In [1]:
import os

os.environ['JAVA_HOME'] = "C:/Program Files/Java/jdk-11"
os.environ['PYSPARK_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['PYSPARK_DRIVER_PYTHON'] = "C:/Users/usr/anaconda3/envs/pyspark_env/python.exe"
os.environ['HADOOP_HOME'] = "C:/hadoop-3.4.0"
os.environ['HADOOP_COMMON_LIB_NATIVE_DIR'] = "C:/hadoop-3.4.0/lib/native"
os.environ['PATH'] += os.pathsep + "C:/hadoop-3.4.0/bin"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

### **Almacenamiento en caché**

- persist() y cache()

- cache() = persist(MEMORY_ONLY)

In [2]:
rdd = sc.parallelize([item for item in range(10)])

In [3]:
from pyspark.storagelevel import StorageLevel

In [4]:
rdd.persist(StorageLevel.MEMORY_ONLY) # Persiste nuestro RDD solo a nivel de memoria.

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

- Si queremos cambiarle el nivel de persistencia a este rdd tenemos antes que hace **unpersist()**

In [5]:
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [6]:
rdd.persist(StorageLevel.DISK_ONLY) # Persiste nuestro RDD solo a nivel de disco.

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [7]:
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

In [8]:
rdd.cache() # Nivel de persistencia solo en memoria.

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

### **Particionado y mezcla de datos(Shuffling)**

<img src="https://github.com/adrianlardies/lab-pyspark/raw/main/images/Shuffling.PNG" width="500"/>

<img src="https://github.com/adrianlardies/lab-pyspark/raw/main/images/Shuffling_2.PNG" width="500"/>